In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd
import datetime as dt

#location of our data
folder_url = 'http://shulok.com/SD_Covid_data/'

#get the contents of the folder
url = requests.get(folder_url).text
soup = BeautifulSoup(url)
print(soup)


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<html>
<head>
<title>Index of /SD_Covid_data</title>
</head>
<body>
<h1>Index of /SD_Covid_data</h1>
<ul><li><a href="/"> Parent Directory</a></li>
<li><a href="COVID_19_Death_Statistics_San_Diego_County.csv"> COVID_19_Death_Statistics_San_Diego_County.csv</a></li>
<li><a href="COVID_19_Hospitalization_Statistics_San_Diego_County.csv"> COVID_19_Hospitalization_Statistics_San_Diego_County.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-10-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-10-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-11-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-11-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-13-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-13-2020.csv</a></li>
<li><a href="tabula-COVID-19%20Deaths%20by%20Demographics%204-17-2020.csv"> tabula-COVID-19 Deaths by Demographics 4-17-2020.c

In [2]:
frames = []
dateformat = "%m-%d-%Y"
last_date = None
for link in soup.findAll("a"):
    current_link = link.get("href")
    if current_link.endswith('csv') and current_link.count('Hospitalizations') == 1:

        #get file date which is right before the file extension
        mylist = current_link.split("%20")
        datestr = mylist[3].split('.')[0]

        #convert date string to date object
        date = dt.datetime.strptime(datestr, dateformat)
        if last_date == None or last_date < date:
            last_date = date
            
        # Read the source file, but use the first column as an index
        df = pd.read_csv(folder_url + current_link, index_col=0)
        
        # Transpose the frame, which puts the row lables as column labels. 
        df = df.T
        
        #The file date is the date when the data was updated
        #It includes data up to the day before
        df['date'] = date - dt.timedelta(days=1)
       
        frames.append(df)
    
df = pd.concat(frames, sort=True) # Concatenate all of the transposed frames

# Move the date column to the front. 
date_col = df.pop('date')
df.insert(0,'date',date_col)

df.sort_values(by=['date'], inplace=True, ascending=True)

df.head(20)




,date,0-9 years,10-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60-69 years,70-79 years,80+ years,...,Gender Unknown,Hispanic or Latino,Male,Multiple Race,Native Hawaiian/Pacific Islander,Pacific Islander,Race/Ethnicity,Race/Ethnicity Other/Unknown,Race/Ethnicity Unknown,White
Count,2020-03-25,0,0,5,11,8,17,10,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-26,0,1,5,13,13,18,10,11,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-27,0,1,5,14,15,21,12,13,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-28,0,1,5,15,15,23,13,15,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-29,0,1,6,17,17,25,15,17,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-30,0,1,8,17,18,27,20,22,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-31,0,1,10,18,19,31,27,27,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-04-01,0,2,10,23,24,34,32,27,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-04-02,0,2,11,25,29,35,41,34,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-04-03,0,2,13,26,30,41,43,36,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
filename = 'COVID_19_Hospitalization_Statistics_San_Diego_County_%s.csv' % last_date.strftime(dateformat)
print(filename)

df.to_csv(filename, index=False)# The index column is useless, so drop it. 
df.head()

COVID_19_Hospitalization_Statistics_San_Diego_County_07-17-2020.csv


,date,0-9 years,10-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60-69 years,70-79 years,80+ years,...,Gender Unknown,Hispanic or Latino,Male,Multiple Race,Native Hawaiian/Pacific Islander,Pacific Islander,Race/Ethnicity,Race/Ethnicity Other/Unknown,Race/Ethnicity Unknown,White
Count,2020-03-25,0,0,5,11,8,17,10,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-26,0,1,5,13,13,18,10,11,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-27,0,1,5,14,15,21,12,13,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-28,0,1,5,15,15,23,13,15,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Count,2020-03-29,0,1,6,17,17,25,15,17,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
